In [1]:
import sys
sys.path.insert(0,'..')
from paths import *
import joblib
import os
import pandas as pd


/Users/karthiksoman/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
p_val_thresh = 0.05


In [3]:
model = []
taxa_list = ['species', 'genus']
for taxa in taxa_list:
    with open(os.path.join(DATA_ROOT_PATH, f'john_hopkins/glm_model_results_for_{taxa}_taxa.joblib'), 'rb') as f:
        model.extend(joblib.load(f)) 
    

In [4]:

full_summary = []
for item in model:
    for index, row in item['model_summary'].iterrows():
        full_summary.append((row['analyte_name'], row['microbial_abundance_coeff'], row['pvalue'], item['microbiome'], item['taxa'], item['cohort_category'], item['sample'], row['number_of_samples']))

full_summary_df = pd.DataFrame(full_summary, columns=['analyte_name', 'microbial_abundance_coeff', 'pvalue', 'microbiome', 'taxa', 'cohort_category', 'sample', 'number_of_samples'])
full_summary_df.sort_values(by='pvalue', inplace=True)


In [5]:

significant_summary = []
for item in model:
    for index, row in item['model_summary'].iterrows():
        if row['pvalue']:
            if (row['pvalue'] < p_val_thresh) & (row['model_converged_flag']==True):
                significant_summary.append((row['analyte_name'], row['microbial_abundance_coeff'], row['pvalue'], item['microbiome'], item['taxa'], item['cohort_category'], item['sample'], row['number_of_samples']))

significant_summary_df = pd.DataFrame(significant_summary, columns=['analyte_name', 'microbial_abundance_coeff', 'pvalue', 'microbiome', 'taxa', 'cohort_category', 'sample', 'number_of_samples'])
significant_summary_df.sort_values(by='pvalue', inplace=True)


In [7]:
full_summary_df.to_excel(os.path.join(DATA_ROOT_PATH, 'john_hopkins', 'microbiome_glm_model_all_associations.xlsx'), index=False, header=True)

significant_summary_df.to_excel(os.path.join(DATA_ROOT_PATH, 'john_hopkins', 'microbiome_glm_model_significant_associations.xlsx'), index=False, header=True)

                    

In [11]:
analyte_df = significant_summary_df[significant_summary_df.analyte_name=='exp_ratio']
analyte_df.reset_index().drop('index', axis=1)


,analyte_name,microbial_abundance_coeff,pvalue,microbiome,taxa,cohort_category,sample,number_of_samples
0,exp_ratio,22.845148,0.000014,Anaerostipes,genus,RRMS,feces,426
1,exp_ratio,-9.924888,0.000424,Bacteroides thetaiotaomicron,species,full,feces,696
2,exp_ratio,617541.924504,0.000568,Dubosiella newyorkensis,species,PMS,feces,180
3,exp_ratio,0.469900,0.001193,Bacteroides,genus,full,serum,696
4,exp_ratio,0.537569,0.001283,Bacteroides,genus,RRMS,serum,516
5,exp_ratio,2.818236,0.047372,Bacteroides thetaiotaomicron,species,full,serum,696


In [9]:
# import pandas as pd
# import matplotlib.pyplot as plt


# # Sort DataFrame for better visualization (optional)
# analyte_df = analyte_df.sort_values(by='microbial_abundance_coeff', ascending=False)

# # Set the figure size
# plt.figure(figsize=(4, 3))

# # Create a bar plot
# bars = plt.barh(analyte_df['microbiome'], analyte_df['microbial_abundance_coeff'], color='skyblue')

# # Annotate each bar with cohort_category and sample
# for bar, cohort, sample in zip(bars, analyte_df['cohort_category'], analyte_df['sample']):
#     plt.text(bar.get_width(), bar.get_y() + bar.get_height()/2,
#              f'{cohort}\n{sample}', ha='center', va='center')

# # Add labels and title
# plt.xlabel('Microbial Abundance Coeff')
# plt.ylabel('Microbiome')
# plt.title('Microbial Abundance Coefficient by Microbiome')

# # Show the plot
# plt.tight_layout()
# plt.show()
